# Conjugate Priors

Think Bayes, Second Edition

Copyright 2020 Allen B. Downey

License: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
# Get utils.py

import os

if not os.path.exists('utils.py'):
    !wget https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py

In [3]:
from utils import set_pyplot_params
set_pyplot_params()


Grid algorithms... three alternative:

1. Conjugate priors

2. ABC

3. MCMC



## Poisson processes

In Chapter xxx, we solved the World Cup problem using a Poisson process to model goals in a soccer game as random events that are equally likely to occur at any point during a game.

We used a gamma distribution to represent the prior distribution of $\lambda$, the goal-scoring rate.  And we used a Poisson distribution to compute the probability of the data, the observed number of goals in a game.

Here's the prior distribution again.

In [5]:
import numpy as np

alpha = 1.4
lams = np.linspace(0, 10, 101)

In [6]:
from utils import make_pmf
from scipy.stats import gamma

dist = gamma(alpha)
prior = make_pmf(dist, lams)

And here's the likelihood of scoring 4 goals for each possible value of `lam`.

In [7]:
from scipy.stats import poisson

k = 4
likelihood = poisson(lams).pmf(k)

We did the update by multiplying the prior by the likelihood and normalizing the result.

In [8]:
posterior = prior * likelihood
posterior.normalize()

And here's the result.

In [14]:
from utils import decorate

def decorate_rate(title=''):
    decorate(xlabel='Goal scoring rate (lam)',
             ylabel='PMF',
             title=title)

In [28]:
prior.plot(label='prior', color='C5')
posterior.plot(label='posterior', color='C1')

decorate_rate('Posterior distribution')

## The conjugate prior

At the time, I said that I chose the gamma distribution for three reasons:

1. The goal scoring rate is continuous and cannot be less than 0, and the gamma distribution is appropriate for this kind of quantity.

2. The gamma distribution has only one parameter, `alpha`, which is the mean.  So it's easy to construct a gamma distribution with the mean we want.

3. The shape of the Gamma distribution is a reasonable choice, given what we know about soccer.

And I said there was a fourth reason that I would reveal later.
Well, now is the time.

The other reason I chose the gamma distribution is that it is the "conjugate prior" of the Poisson distribution, so-called because the two distributions are connected or coupled, which is what "conjugate" means.

In the next section I'll explain *how* they are connected, but first I'll show you the consequence of this connection, which is that there is a remarkably simple way to compute the posterior distribution.

However, in order to demonstrate it, we have to switch from the one-parameter version of the gamma distribution to the two-parameter version.  Since the first parameter is called `alpha`, you might guess that the second parameter is called `beta`.

The following function takes `alpha` and `beta` and makes an object that represents a gamma distribution with those parameters.

In [18]:
def make_gamma_dist(alpha, beta):
    """Makes a gamma object.
    
    alpha: shape parameter
    beta: scale parameter
    
    returns: gamma object
    """
    dist = gamma(alpha, scale=1/beta)
    dist.alpha = alpha
    dist.beta = beta
    return dist

Here's the prior distribution with `alpha=1.4` again and `beta=1`. 

In [19]:
alpha = 1.4
beta = 1

prior_gamma = make_gamma_dist(alpha, beta)
prior_gamma.mean()

Now to do a Bayesian update, I claim, all we have to do is make a gamma distribution with parameters `alpha+k` and `beta+1`.

In [23]:
def update_gamma(prior, data):
    """Update a gamma prior.
    
    prior: Pmf
    data: tuple of (k goals, t games)
    
    returns: scipy.stats.dist
    """
    k, t = data
    alpha = prior.alpha + k
    beta = prior.beta + t
    return make_gamma_dist(alpha, beta)

Here's how we update it with the data, which is 4 goals in 1 game.

In [24]:
data = 4, 1
posterior_gamma = update_gamma(prior_gamma, data)

After all the work we did in Chapter 7, it might seem absurd that we can do a Bayesian update by adding two pairs of numbers.
So let's confirm that it works.

I'll evaluate the posterior distribution for each quantity in `lams` and put the results in a `Pmf` object.

In [25]:
posterior_conjugate = make_pmf(posterior_gamma, lams)

So `posterior_conjugate` contains a discrete approximation of the posterior distribution we just computed using the conjugate prior.
The following figure shows the result along with the posterior we computed using the grid algorithm.

In [29]:
posterior.plot(label='grid posterior', color='C1')
posterior_conjugate.plot(label='conjugate posterior', 
                         color='C4', linestyle='dotted')

decorate_rate('Posterior distribution')

They are the same other than small differences due to floating-point approximations.

In [30]:
np.allclose(posterior, posterior_conjugate)

## What the Actual?

To understand how that works, we'll write the PDF of the gamma prior and the PMF of the Poisson likelihood, then multiply them together, because that's what the Bayesian update does.
We'll see that the result is a gamma distribution, and we'll derive its parameters.

Here's the PDF of the gamma prior, which is the probability density for each value of $\lambda$, given parameters $\alpha$ and $\beta$:

$$\lambda^{\alpha-1}e^{-\beta \lambda}$$

I have omitted the normalizing factor; since we are planning to normalize the posterior distribution anyway, we don't really need it.

Now suppose a team scores $k$ goals in $t$ games.
The probability of this data is given by PMF of the Poisson distribution, which is a function of $k$ with $\lambda$ and $t$ as a parameter.

$$\lambda^k e^{-\lambda t}$$

Again, I have omitted the normalizing factor, which makes it clearer that the gamma and Poisson distributions have the same functional form.
When we multiply them together, we can pair up the factors and add up the exponents.
The result is the unnormalized posterior distribution,

$$\lambda^{\alpha-1+k} e^{-(\beta + t) \lambda}$$

which we can recognize as an unnormalized gamma distribution with parameters $\alpha + k$ and $\beta + t$.

This derivation provides insight into what the parameters of the posterior distribution mean: $\alpha$ reflects the number of events that have occurred; $\beta$ reflects the elapsed time.

## Binomial likelihood

As a second example, let's look again at the Euro problem.
When we solved it with a grid algorithm, we started with a uniform prior:

In [36]:
from utils import make_uniform

xs = np.linspace(0, 1, 101)
uniform = make_uniform(xs, 'uniform')

We used the binomial distribution to compute the likelihood of the data, which was 140 heads out of 250 attempts.

In [37]:
from scipy.stats import binom

k, n = 140, 250
xs = uniform.qs

likelihood = binom.pmf(k, n, xs)

Then we computed the posterior distribution in the usual way.

In [38]:
posterior = uniform * likelihood
posterior.normalize()

We can solve this problem more efficiently using the conjugate prior of the binomial distribution, which is the beta distribution.

The beta distribution is bounded between 0 and 1, so it works well for representing the distribution of a probability like `x`.
It has two parameters, called `alpha` and `beta`, that determine the shape of the distribution.

SciPy provides an object called `beta` that represents a beta distribution.
The following function takes `alpha` and `beta` and returns a new `beta` object.

In [39]:
import scipy.stats

def make_beta(alpha, beta):
    """Makes a beta object.
    
    alpha: shape parameter
    beta: scale parameter
    
    returns: scipt.stats.beta object
    """
    dist = scipy.stats.beta(alpha, beta)
    dist.alpha = alpha
    dist.beta = beta
    return dist

It turns out that the uniform distribution, which we used as a prior, is the beta distribution with parameters `alpha=1` and `beta=1`.
So we can make a `beta` object that represents a uniform distribution, like this:

In [40]:
alpha = 1
beta = 1

prior_beta = make_beta(alpha, beta)
prior_beta.mean()

Now let's figure out how to do the update.  As in the previous example, we'll write the PDF of the prior distribution and the PMF of the likelihood function, and multiply them together.  We'll see that the product has the same form as the prior, and we'll derive its parameters.

Here is the PDF of the beta distribution, which is a function of $x$ with $\alpha$ and $\beta$ as parameters.

$$x^{\alpha-1} (1-x)^{\beta-1}$$

Again, I have omitted the normalizing factor, which we don't need because we are going to normalize the distribution after the update.

The beta PDF might look familiar: the PMF of the binomial distribution has the same form, but we interpret it as a function of $k$ with $n$ and $x$ as parameters.

$$x^{k} (1-x)^{n-k}$$

Again, I have omitted the normalizing factor.
Now when we multiply the beta prior and the binomial likelihood, the result is

$$x^{\alpha-1+k} (1-x)^{\beta-1+n-k}$$

which we recognize as an unnormalized beta distribution with parameters $\alpha+k$ and $\beta+n-k$.

So, to update the beta prior, all we have to do is make a beta distribution with parameters `alpha+k` and `beta+n-k`, as in the following function.

In [41]:
def update_beta(prior, data):
    """Update a beta distribution.
    
    prior: SciPy beta object
    data: number of successes and number of trials (k, n)
    
    returns: SciPy beta object
    """
    k, n = data
    alpha = prior.alpha + k
    beta = prior.beta + n - k
    return make_beta(alpha, beta)

Again, the conjugate prior gives us insight into the meaning of the parameters; $\alpha$ is related to the number of observed successes; $\beta$ is related to the number of failures.

Here's how we do the update with the observed data.

In [42]:
data = 140, 250
posterior_beta = update_beta(prior_beta, data)
posterior_beta.mean()

To confirm that it works, I'll evaluate the posterior distribution for the possible values of `xs` and put the results in a `Pmf`.

In [43]:
posterior_conjugate = make_pmf(posterior_beta, xs)

And we can compare the posterior distribution we just computed with the results from the grid algorithm.

In [44]:
def decorate_euro(title):
    decorate(xlabel='Proportion of heads (x)',
             ylabel='Probability',
             title=title)

In [45]:
posterior.plot(label='grid posterior', color='C1')
posterior_conjugate.plot(label='conjugate posterior',
                        color='C4', linestyle='dotted')

decorate_euro(title='Posterior distribution of x')

They are the same other than small differences due to floating-point approximations.

In [46]:
np.allclose(posterior, posterior_conjugate)

## Lions and tigers and bears

Suppose we visit a wild animal preserve where we know that the only animals are lions and tigers and bears, but we don't know how many of each there are.

During the tour, we see 3 lions, 2 tigers, and one bear. Assuming that every animal had an equal chance to appear in our sample, what is the probability that the next animal we see is a bear?

To answer this question, we'll use the data to estimate the prevalence of each species, that is, what fraction of the animals belong to each species.

If we know the prevalences, we can use the multinomial distribution to compute the probability of the data.
For example, suppose we know that the fraction of lions, tigers, and bears is 0.4, 0.3, and 0.3, respectively.

In that case the probability of the data is:

In [47]:
from scipy.stats import multinomial

data = np.array([3, 2, 1])
n = data.sum()
ps = 0.4, 0.3, 0.3

multinomial.pmf(data, n, ps)

Now, we could choose a prior for the prevalences and do a Bayesian update using the multinomial distribution to compute the probability of the data.

But there's an easier way, because the multinomial distribution has a conjugate prior: the Dirichlet distribution.

## The Dirichlet distribution

The Dirichlet distribution is a multivariate distribution, like the multivariate normal distribution we used in Chapter xxx to describe the distribution of penguin measurements.  

In that example, the quantities in the distribution are pairs of flipper length and culmen length, and the parameters of the distribution are a vector of means and a matrix of covariances.

In a Dirichlet distribution, the quantities are vectors of probabilities, $\pmb{x}$, and the parameter is a vector, $\pmb{\alpha}$.

An example will make that clearer.  SciPy provides a `dirichlet` object that represents a Dirichlet distribution.
Here's an instance with $\pmb{\alpha} = 1, 2, 3$.

In [49]:
from scipy.stats import dirichlet

alpha = np.array([1, 2, 3])
dist = dirichlet(alpha)

Since we provided three parameters, the result is a distribution of three variables.
If we draw a random value from this distribution, like this:

In [50]:
dist.rvs()

The result is an array of three values.

The values are bounded between 0 and 1, so they can be interpreted as probabilities.
Even better, they always add up to 1, so they can be interpreted as the probabilities of a set of outcomes that are mutually exclusive and collectively exhaustive.

In [51]:
dist.rvs().sum()

Let's see what the distributions of these values look like.  I'll draw 1000 random vectors from this distribution, like this:

In [52]:
sample = dist.rvs(1000)
sample.shape

The result is an array with 1000 rows and three columns.  I'll compute the `Cdf` of the values in each column.

In [54]:
from empiricaldist import Cdf

cdfs = [Cdf.from_seq(col) for col in sample.transpose()]

These are the marginal distributions of the three variables.  Here's what they look like.

In [56]:
for i, cdf in enumerate(cdfs):
    label = f'Column {i}'
    cdf.plot(label=label)
    
decorate()

Column 0, which corresponds to the lowest parameter, contains the lowest probabilities.
Column 2, which corresponds to the highest parameter, contains the highest probabilities.

We can compute the means of the marginal distributions like this:

In [57]:
alpha / alpha.sum()

And the `dist` object can do it for us.

In [58]:
dist.mean()

Let's compare those to the means of the samples.

In [59]:
sample.mean(axis=0)

That's pretty close.

As it turns out, these marginal distributions are beta distributions.
The following function takes the vector of parameters, `alpha`, and computes the marginal distribution of variable `i`:

In [61]:
def marginal_beta(alpha, i):
    """Compute the ith marginal of a Dirichlet distribution.
    
    alpha: vector parameter of the distribution
    i: index of the marginal
    """
    total = alpha.sum()
    return make_beta(alpha[i], total-alpha[i])

The following plot uses this function to plot the marginal beta distributions and compares them to the distributions of the columns in the sample.

In [64]:
xs = np.linspace(0, 1, 101)

for i, cdf in enumerate(cdfs):
    label = f'Column {i}'
    
    marginal = marginal_beta(alpha, i)
    make_pmf(marginal, xs).make_cdf().plot(color='C5')
    cdf.plot(label=label, linestyle='dotted')

decorate()

So that checks out.

This time I will not derive the relationship between the Dirichlet prior and the multinomial likelihood function, but I will tell you how to use this relationship to do a Bayesian update.

If the prior distribution is Dirichlet with parameter vector `alpha` and the data is a vector of observations, `data`, the posterior distribution is Dirichlet with parameter vector `alpha + data`.

As an exercise at the end of this chapter, you can use this method to solve the Lions and Tigers and Bears problem.

## Summary

After reading this chapter, if you feel like you've been tricked, I understand.  It turns out that many of the problems in this book can be solved, using conjugate priors, with just a few arithmetic operations.  So why did we go to all the trouble of using grid algorithms?

Sadly, there are only a few problems we can solve with conjugate priors; in fact, this chapter includes most of the ones that are useful in practice.

For most problems, there is no conjugate prior and no shortcut to compute the posterior distribution.  That's why we need grid algorithms and the methods in the next two chapters, approximate Bayesian computation (ABC) and Markov chain Monte Carlo methods (MCMC).

## Exercises

**Exercise:** In the second version of the World Cup problem, the data we use for the update is not the number of goals in a game, but the time until the first goal.
So the probability of the data is given by the exponential distribution rather than the Poisson distribution. 

But it turns out that the gamma distribution is *also* the conjugate prior of the exponential distribution, so there is a simple closed-form way to compute this update, too.

The PDF of the exponential distribution is a function of $t$ with $\lambda$ as a parameter.

$$\lambda e^{-\lambda t}$$

Multiply the PDF of the gamma prior by this likelihood, confirm that the result is an unnormalized gamma distribution, and see if you can derive its parameters.

Write a few lines of code to update `prior_gamma` with the data from this version of the problem, which was a first goal after 11 minutes and a second goal after an additional 12 minutes.

Remember to express these quantities in units of games, which are approximately 90 minutes.

In [65]:
# Solution goes here

In [67]:
# Solution goes here

In [68]:
# Solution goes here

In [69]:
# And here's what the posteriors look like

make_pmf(prior_gamma, lams).plot(color='C5', label='prior')
make_pmf(posterior1, lams).plot(label='after 1 goal')
make_pmf(posterior2, lams).plot(label='after 2 goals')

decorate_rate(title='World Cup Problem, Germany v Brazil')

**Exercise:** For problems like the Euro problem where the likelihood function is binomial, we can do a Bayesian update with just a few arithmetic operations, but only if the prior is a beta distribution.

As we saw, the uniform distribution is a beta distribution with particular parameters, but what can we do if the prior distribution we want is not a beta distribution?  For example, in Chapter xxx we also solved the Euro problem with a triangle prior, and a beta distribution cannot be a triangle.

Even so, we can often find a beta distribution that is a good-enough approximation for the prior we want.

Here's the triangle prior again.

In [71]:
from empiricaldist import Pmf

ramp_up = np.arange(50)
ramp_down = np.arange(50, -1, -1)

a = np.append(ramp_up, ramp_down)
xs = uniform.qs

triangle = Pmf(a, xs, name='triangle')
triangle.normalize()

And here's the update.

In [72]:
k, n = 140, 250
likelihood = binom.pmf(k, n, xs)

posterior = triangle * likelihood
posterior.normalize()

See if you can find a beta distribution that fits the triangle prior, then update it using `update_beta`.

Use `make_pmf` to make a `Pmf` that approximates the posterior distribution and compare it to the posterior we just computed using a grid algorithm.  How big is the largest difference between them?

To get you started, here's the beta distribution that we used as a uniform prior.

In [73]:
alpha = 1
beta = 1

prior_beta = make_beta(alpha, beta)
prior_beta.mean()

And here's what it looks like compared to the triangle prior.

In [74]:
prior_pmf = make_pmf(prior_beta, xs)

triangle.plot(label='triangle')
prior_pmf.plot(label='beta')

decorate_euro('Prior distributions')

In [75]:
# Solution goes here

In [76]:
# Solution goes here

In [78]:
# Solution goes here

In [82]:
# Solution goes here

**Exercise:**  [3Blue1Brown](https://en.wikipedia.org/wiki/3Blue1Brown) is a YouTube channel about math; if you are not already aware of it, I recommend it highly.

In [this video](https://www.youtube.com/watch?v=8idr1WZ1A7Q) the narrator presents this problem:

> You are buying a product online and you see three sellers offering the same product at the same price.  One of them has a 100% positive rating, but with only 10 reviews.  Another has a 96% positive rating with 50 total reviews.  And yet another has a 93% positive rating, but with 200 total reviews.
>
>Which one should you buy from?

Let's think about how to model this scenario.  Suppose each seller has some unknown probability, `x`, of providing satisfactory service and getting a positive rating, and we want to choose the seller with the highest value of `x`.

This is not the only model for this scenario, and it is not necessarily the best.  An alternative would be something like item response theory, where sellers have varying ability to provide satisfactory service and customers have varying difficulty of being satisfied.

But the first model has the virtue of simplicity, so let's see where it gets us.

As a prior, I suggest we use a beta distribution with `alpha=8` and `beta=2`.  Here's what it looks like:

In [84]:
prior = make_beta(8, 2)

xs = np.linspace(0.005, 0.995, 199)
prior_pmf = make_pmf(prior, xs)
prior_pmf.plot(color='C5', label='prior')

decorate(xlabel='Probability of positive rating',
         ylabel='PDF')

This prior implies that most sellers are satisfactory most of the time, but none are perfect.

1. Use the data to update this prior for the three sellers and plot the posterior distributions.  Which seller has the highest posterior mean?

2. How confident should we be about our choice?  That is, what is the probability that the seller with the highest posterior mean actually has the highest value of `x`?

3. Consider an alternative prior with `alpha=0.7` and `beta=0.5`.  What does this prior look like and what does it imply about sellers?

4. Run the analysis again with the alternative prior and see what effect it has on the results.

You might notice that I restricted the range of `xs` so it does not include 0 and 1.  I did that because when the parameters of the beta distribution are less than 1, the probability density goes to infinity at 0 and 1.  From a mathematical point of view, that's not a problem; it is still a proper probability distribution.  But from a computational point of view, it means we have to avoid evaluating the PDF at 0 and 1. 

In [85]:
# Solution goes here

In [86]:
# Solution goes here

In [87]:
# Solution goes here

In [90]:
# Solution goes here

In [91]:
# Solution goes here

In [92]:
# Solution goes here

In [93]:
# Solution goes here

**Exercise:** Use a Dirichlet prior with parameter vector `alpha = [1, 1, 1]` to solve the Lions and Tigers and Bears problem:

>Suppose we visit a wild animal preserve where we know that the only animals are lions and tigers and bears, but we don't know how many of each there are.
>
>During the tour, we see 3 lions, 2 tigers, and one bear. Assuming that every animal had an equal chance to appear in our sample, estimate the prevalence of each species.
>
>What is the probability that the next animal we see is a bear?


In [94]:
# Solution goes here

In [95]:
# Solution goes here

In [96]:
# Solution goes here

In [98]:
# Solution goes here